In [9]:
import torch
import yaml

# Local imports
import utils.carl_utils.preprocessing as carl_ppro
import utils.carl_utils.train as carl_train

import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#DEVICE = 'cpu'
print(DEVICE)

cuda:0


In [2]:
total_features = yaml.load(open("carl_features.yaml", 'r'), Loader=yaml.CLoader)
total_features

{'Electron_set': {'set': True,
  'size': 4,
  'subfeatures': ['Electron_pt',
   'Electron_eta',
   'Electron_phi',
   'Electron_mass']},
 'FatJet_set': {'set': True,
  'size': 5,
  'subfeatures': ['FatJet_pt',
   'FatJet_eta',
   'FatJet_phi',
   'FatJet_mass',
   'FatJet_btagCSVV2']},
 'Jet_set': {'set': True,
  'size': 5,
  'subfeatures': ['Jet_pt',
   'Jet_eta',
   'Jet_phi',
   'Jet_mass',
   'Jet_btagCSVV2']},
 'MET_sumEt': {'set': False, 'size': 1, 'subfeatures': ['MET_sumEt']},
 'Muon_set': {'set': True,
  'size': 4,
  'subfeatures': ['Muon_pt', 'Muon_eta', 'Muon_phi', 'Muon_mass']},
 'Photon_set': {'set': True,
  'size': 4,
  'subfeatures': ['Photon_pt', 'Photon_eta', 'Photon_phi', 'Photon_mass']}}

In [3]:
subfeatures = ["Jet_set"]#, "Electron_set", "Muon_set"]
features = dict(zip(subfeatures, [total_features[feat] for feat in subfeatures]))
features

{'Jet_set': {'set': True,
  'size': 5,
  'subfeatures': ['Jet_pt',
   'Jet_eta',
   'Jet_phi',
   'Jet_mass',
   'Jet_btagCSVV2']}}

In [4]:
weight_features = ["genWeight", "btagWeight_CSVV2"]

## Initiate the datasets and load the data

In [5]:
n_train_events = int(1e6) // 2
n_val_events = int(2e5) // 2

train_nominal_dataset = carl_ppro.DeepSetsDataset(["/data/mdrnevich/AGC/CMS_ttbar_nominal_DeepSets_training_data_new2.root",], features, 0, stop_event=n_train_events)
valid_nominal_dataset = carl_ppro.DeepSetsDataset(["/data/mdrnevich/AGC/CMS_ttbar_nominal_DeepSets_validation_data_new2.root",], features, 0, stop_event=n_val_events)

train_PS_var_dataset = carl_ppro.DeepSetsDataset(["/data/mdrnevich/AGC/CMS_ttbar_PS_var_DeepSets_training_data_new2.root",], features, 1, stop_event=n_train_events)
valid_PS_var_dataset = carl_ppro.DeepSetsDataset(["/data/mdrnevich/AGC/CMS_ttbar_PS_var_DeepSets_validation_data_new2.root",], features, 1, stop_event=n_val_events)

100%|██████████| 100000/100000 [00:24<00:00, 4138.42it/s]


## Prepare the data

In [6]:
train_generator_data = carl_ppro.CombinedDataset(train_nominal_dataset, train_PS_var_dataset)
valid_generator_data = carl_ppro.CombinedDataset(valid_nominal_dataset, valid_PS_var_dataset)

## Define the settings of the model and training

In [7]:
hidden1 = 512
hidden2 = 256
hidden3 = 128
classify1 = 1024
classify2 = 1024
outputs = 1

model_settings = {
    "features": features,
    "phi": [hidden1, hidden2, hidden3],
    "mlp": [classify1, classify2]
}


training_settings = {
    "optimizer": "Adam",
    "learning_rate": 1e-2,
    "batch_size": 256,
    "n_epochs": 30,
    "patience": 6,
    "device": DEVICE,
    "n_events": n_train_events * 2,
    "saveAs": "models/deepsets_agc_model_jets"
}

# Perform the training

In [ ]:
model = carl_train.train(model_settings, train_generator_data, valid_generator_data, **training_settings)

Constructing the model
Loading the input data scaling


100%|██████████| 977/977 [00:06<00:00, 141.20it/s]


Training the model


  3%|▎         | 1/30 [02:22<1:08:58, 142.72s/it]                  

Epoch: 01, Training Loss:   0.6875
           Validation Loss: 0.6826
New best model saved to: models/deepsets_agc_model_jets.zip


...

 40%|████      | 12/30 [27:20<40:39, 135.52s/it]                   

Epoch: 12, Training Loss:   0.6800
           Validation Loss: 0.6811
Stale epoch


 43%|████▎     | 13/30 [29:34<38:17, 135.17s/it]                   

Epoch: 13, Training Loss:   0.6798
           Validation Loss: 0.6801
New best model saved to: models/deepsets_agc_model_jets.zip


loss = 0.66905:  75%|███████▌  | 2931/3907 [01:34<00:29, 32.71it/s]

In [10]:
model

DeepSetsEnsemble(
  (phi0):  Sequential(
    (0): Conv1d(5, 512, kernel_size=(1,), stride=(1,))
    (1): ReLU()
    (2): Conv1d(512, 256, kernel_size=(1,), stride=(1,))
    (3): ReLU()
    (4): Conv1d(256, 128, kernel_size=(1,), stride=(1,))
    (5): ReLU()
  )
  (mlp):  Sequential(
    (0): Linear(in_features=128, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=1024, out_features=1024, bias=True)
    (4): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=1024, out_features=1, bias=True)
    (7): Sigmoid()
  )
)